In [67]:
import requests
import pandas as pd
import numpy as np
import kagglehub
import os
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler

Data Acquisition

In [68]:
path = kagglehub.dataset_download("blastchar/telco-customer-churn",path='WA_Fn-UseC_-Telco-Customer-Churn.csv')

100%|██████████| 955k/955k [00:00<00:00, 1.81MB/s]


In [69]:
destination = os.environ.get('LR_Destination')

alr_exists = str(destination + '/' + 'WA_Fn-UseC_-Telco-Customer-Churn.csv')

if os.path.exists(alr_exists):
        os.remove(alr_exists)
        print(f"File '{alr_exists}' deleted sucessfully.")
else:
        print(f"File '{alr_exists}' not found.")

File 'C:/Users/sarge/Documents/Projects/Covid Visualization/covid-dashboard-analytics/data/WA_Fn-UseC_-Telco-Customer-Churn.csv' deleted sucessfully.


In [70]:
source = os.environ.get('LR_SOURCE')

allfiles = os.listdir(source)

for f in allfiles:
        src_path = os.path.join(source, f)
        dst_path = os.path.join(destination, f)
        os.rename(src_path, dst_path)

In [71]:
str(destination + '/' + 'WA_Fn-UseC_-Telco-Customer-Churn.csv')  
ChurnData = pd.read_csv(str(destination + '/' + 'WA_Fn-UseC_-Telco-Customer-Churn.csv'))

Data Proccessing

In [27]:
#Pre-Processing of Data, removing CustomerID, and Converting TotalCharges into a float.
ChurnData.drop('customerID',axis=1,inplace=True)
ChurnData['TotalCharges'] = pd.to_numeric(ChurnData['TotalCharges'], errors='coerce').fillna(pd.to_numeric(ChurnData['TotalCharges'], errors='coerce').mean())

In [28]:
# most likely the columns that are categorical -> not numerical (Creating function to dynamically assign integer values to categorical information)
# Some data is multicollinear, meaning that there is redundant information.
for column in ChurnData:
    if(ChurnData[column].dtype == 'object' and len(ChurnData[column].unique()) > 2):
            ChurnData = pd.get_dummies(ChurnData, columns=[str(column)], drop_first=True)

#Convert booleans created from pd.get_dummies and map them dynamically into 0s and 1s
for column in ChurnData:
      if(ChurnData[column].dtypes == bool):
            ChurnData[column] = ChurnData[column].astype(int)

In [29]:
#Retrieving columns that were generated by pandas dummy variables and calculating VIF for detecting colinearity
data = ChurnData.loc[:, ChurnData.columns.str.contains("_")]

vif_df = pd.DataFrame()
vif_df["feature"] = data.columns
vif_df["VIF_Score"] = [variance_inflation_factor(data.values, i) for i in range(len(data.columns))]

#Retrieving scores that were greater than 10, indicates need to be dropped from original dataset
vif_df = vif_df[vif_df["VIF_Score"] > 10]['feature']

c:\Users\sarge\Documents\Projects\Covid Visualization\covid-dashboard-analytics\VirtualEnv\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


In [30]:
cols = [col for col in ChurnData.columns if col not in vif_df.to_numpy()]
ChurnData = ChurnData.loc[:, cols]

In [31]:
#Mapping remaining yes/no columns to 0s and 1s dynamically
yes_no_map = {'Yes' : 1, 'No' : 0}

map_yes_no_columns = []
for cols in ChurnData:
    if 'Yes' in ChurnData[cols].unique() or 'No' in ChurnData[cols].unique():
        map_yes_no_columns.append(ChurnData[cols].name)

for row in ChurnData[map_yes_no_columns]:
    ChurnData[row] = ChurnData[map_yes_no_columns][row].map(yes_no_map)

ChurnData.rename(columns={'gender' : 'Male'}, inplace=True)

In [32]:
gender_map = {'Male': 1, 'Female': 0}

ChurnData['Male'] = ChurnData["Male"].map(gender_map)


Logistic Regression Model Building

In [59]:
ChurnData_X, ChurnData_y = ChurnData.loc[:,ChurnData.columns != 'Churn'], ChurnData['Churn']

CD_X_train, CD_X_test, CD_y_train, CD_y_test = train_test_split(
    ChurnData_X, ChurnData_y, test_size=0.2, random_state=42
)

scaler = StandardScaler()

CD_X_train = scaler.fit_transform(CD_X_train)
CD_X_test = scaler.fit_transform(CD_X_test)

ChurnModel = LogisticRegression()
ChurnModel.fit(CD_X_train, CD_y_train)

LogisticRegression()

Metrics

In [62]:
churn_pred = ChurnModel.predict(CD_X_test)
accuracy = accuracy_score(CD_y_test, churn_pred)

print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 82.11%


In [74]:
print("\nClassification Report:\n", classification_report(CD_y_test, churn_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.90      0.88      1036
           1       0.68      0.61      0.64       373

    accuracy                           0.82      1409
   macro avg       0.77      0.75      0.76      1409
weighted avg       0.82      0.82      0.82      1409

